In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# AI Agent Negotiation Simulator - Interactive Tutorial\n",
    "\n",
    "This notebook demonstrates how to use the negotiation simulator for various scenarios."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from pathlib import Path\n",
    "\n",
    "# Import simulator modules\n",
    "from models import (\n",
    "    Entity, Issue, UtilityFunction, NegotiationPolicy,\n",
    "    PolicyType, PolicyParameters, SimulationConfig\n",
    ")\n",
    "from protocol import NegotiationEngine, BatchNegotiationRunner\n",
    "from utilities import analyze_negotiation_space, find_nash_bargaining_solution\n",
    "from advisor import NegotiationAdvisor, analyze_and_improve\n",
    "from visualization import (\n",
    "    plot_utility_progression, plot_offer_space, \n",
    "    plot_pareto_frontier, plot_batch_success_analysis\n",
    ")\n",
    "\n",
    "print(\"Negotiation Simulator loaded successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Simple Two-Party Trade Negotiation\n",
    "\n",
    "Let's start with a basic buyer-seller negotiation over price and quantity."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the issues\n",
    "issues = [\n",
    "    Issue(name=\"price\", min_value=1000, max_value=5000, unit=\"USD\"),\n",
    "    Issue(name=\"quantity\", min_value=100, max_value=1000, unit=\"units\")\n",
    "]\n",
    "\n",
    "# Create buyer (wants low price, high quantity)\n",
    "buyer = Entity(\n",
    "    name=\"Buyer\",\n",
    "    utility_function=UtilityFunction(\n",
    "        weights={\"price\": 0.7, \"quantity\": 0.3},\n",
    "        ideal_values={\"price\": 1000, \"quantity\": 1000},\n",
    "        reservation_values={\"price\": 4000, \"quantity\": 200}\n",
    "    ),\n",
    "    policy=NegotiationPolicy(\n",
    "        type=PolicyType.LINEAR_CONCESSION,\n",
    "        params=PolicyParameters(\n",
    "            accept_threshold=0.65,\n",
    "            concession_rate=0.08\n",
    "        )\n",
    "    )\n",
    ")\n",
    "\n",
    "# Create seller (wants high price, low quantity)\n",
    "seller = Entity(\n",
    "    name=\"Seller\",\n",
    "    utility_function=UtilityFunction(\n",
    "        weights={\"price\": 0.8, \"quantity\": 0.2},\n",
    "        ideal_values={\"price\": 5000, \"quantity\": 100},\n",
    "        reservation_values={\"price\": 2000, \"quantity\": 800}\n",
    "    ),\n",
    "    policy=NegotiationPolicy(\n",
    "        type=PolicyType.TIT_FOR_TAT,\n",
    "        params=PolicyParameters(\n",
    "            accept_threshold=0.6,\n",
    "            stubbornness=0.4\n",
    "        )\n",
    "    )\n",
    ")\n",
    "\n",
    "print(\"Entities created:\")\n",
    "print(f\"- {buyer.name}: {buyer.policy.type.value} strategy\")\n",
    "print(f\"- {seller.name}: {seller.policy.type.value} strategy\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Analyze the Negotiation Space"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze if agreement is possible\n",
    "analysis = analyze_negotiation_space([buyer, seller], issues, samples=500)\n",
    "\n",
    "print(f\"Has ZOPA (Zone of Possible Agreement): {analysis['has_zopa']}\")\n",
    "if analysis['has_zopa']:\n",
    "    print(f\"ZOPA size: {analysis['zopa_size']} possible agreements\")\n",
    "    print(f\"Max joint utility: {analysis['max_joint_utility']:.3f}\")\n",
    "    \n",
    "    # Find Nash bargaining solution\n",
    "    nash = analysis['nash_solution']\n",
    "    print(f\"\\nNash Bargaining Solution:\")\n",
    "    for issue, value in nash.items():\n",
    "        print(f\"  {issue}: {value:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Run the Negotiation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create configuration\n",
    "config = SimulationConfig(\n",
    "    entities=[buyer, seller],\n",
    "    issues=issues,\n",
    "    max_rounds=50,\n",
    "    protocol=\"alternating\"\n",
    ")\n",
    "\n",
    "# Run negotiation\n",
    "engine = NegotiationEngine(config)\n",
    "outcome = engine.run()\n",
    "\n",
    "# Display results\n",
    "print(outcome.summary())\n",
    "\n",
    "if outcome.success:\n",
    "    print(\"\\nFinal Agreement:\")\n",
    "    for issue, value in outcome.final_agreement.items():\n",
    "        print(f\"  {issue}: {value:.2f}\")\n",
    "    \n",
    "    print(\"\\nUtility Scores:\")\n",
    "    for entity, utility in outcome.final_utilities.items():\n",
    "        print(f\"  {entity}: {utility:.3f} ({utility*100:.1f}% satisfaction)\")\n",
    "else:\n",
    "    print(f\"\\nImpasse reason: {outcome.impasse_reason}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Visualize the Negotiation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot utility progression\n",
    "plot_utility_progression(outcome)\n",
    "\n",
    "# Plot offer trajectory in 2D space\n",
    "plot_offer_space(outcome, \"price\", \"quantity\")\n",
    "\n",
    "# Plot Pareto frontier\n",
    "plot_pareto_frontier([buyer, seller], issues, highlight_outcome=outcome)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Get AI Advisory Recommendations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create advisor\n",
    "advisor = NegotiationAdvisor()\n",
    "\n",
    "# Analyze the outcome\n",
    "report = advisor.analyze_outcome(config, outcome)\n",
    "\n",
    "print(\"Advisory Report\")\n",
    "print(\"=\"*50)\n",
    "print(f\"\\nAnalysis: {report.outcome_analysis}\")\n",
    "\n",
    "print(f\"\\nSuccess Probability: {report.success_probability:.1%}\")\n",
    "\n",
    "if report.parameter_suggestions:\n",
    "    print(\"\\nParameter Suggestions:\")\n",
    "    for suggestion in report.parameter_suggestions[:3]:  # Top 3\n",
    "        print(f\"  {suggestion.entity_name}.{suggestion.parameter_path}:\")\n",
    "        print(f\"    Current: {suggestion.current_value:.2f}\")\n",
    "        print(f\"    Suggested: {suggestion.suggested_value:.2f}\")\n",
    "        print(f\"    Rationale: {suggestion.rationale}\")\n",
    "        print(f\"    Confidence: {suggestion.confidence:.1%}\")\n",
    "\n",
    "if report.key_insights:\n",
    "    print(\"\\nKey Insights:\")\n",
    "    for insight in report.key_insights:\n",
    "        print(f\"  • {insight}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Batch Analysis - Testing Different Strategies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test different policy combinations\n",
    "policy_combinations = [\n",
    "    (PolicyType.LINEAR_CONCESSION, PolicyType.LINEAR_CONCESSION),\n",
    "    (PolicyType.LINEAR_CONCESSION, PolicyType.TIT_FOR_TAT),\n",
    "    (PolicyType.BOULWARE, PolicyType.CONCEDER),\n",
    "    (PolicyType.TIT_FOR_TAT, PolicyType.TIT_FOR_TAT),\n",
    "]\n",
    "\n",
    "results_summary = []\n",
    "\n",
    "for buyer_policy, seller_policy in policy_combinations:\n",
    "    # Update policies\n",
    "    buyer.policy.type = buyer_policy\n",
    "    seller.policy.type = seller_policy\n",
    "    \n",
    "    # Run batch\n",
    "    config = SimulationConfig(\n",
    "        entities=[buyer, seller],\n",
    "        issues=issues,\n",
    "        max_rounds=50\n",
    "    )\n",
    "    \n",
    "    runner = BatchNegotiationRunner(config)\n",
    "    results = runner.run_batch(20)\n",
    "    analysis = runner.analyze_results()\n",
    "    \n",
    "    results_summary.append({\n",
    "        'Buyer Policy': buyer_policy.value,\n",
    "        'Seller Policy': seller_policy.value,\n",
    "        'Success Rate': f\"{analysis['success_rate']:.1%}\",\n",
    "        'Avg Rounds': f\"{analysis['average_rounds']:.1f}\"\n",
    "    })\n",
    "\n",
    "# Display comparison\n",
    "df = pd.DataFrame(results_summary)\n",
    "print(\"\\nPolicy Combination Performance:\")\n",
    "print(df.to_string(index=False))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Multi-Party Resource Allocation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a 3-party negotiation over shared resources\n",
    "resource_issues = [\n",
    "    Issue(name=\"water\", min_value=0, max_value=100, unit=\"million_gallons\"),\n",
    "    Issue(name=\"energy\", min_value=0, max_value=100, unit=\"GWh\"),\n",
    "    Issue(name=\"land\", min_value=0, max_value=100, unit=\"sq_km\")\n",
    "]\n",
    "\n",
    "# Create three countries with different priorities\n",
    "countries = [\n",
    "    Entity(\n",
    "        name=\"Agricultural_Nation\",\n",
    "        utility_function=UtilityFunction(\n",
    "            weights={\"water\": 0.6, \"energy\": 0.1, \"land\": 0.3},\n",
    "            ideal_values={\"water\": 60, \"energy\": 20, \"land\": 50},\n",
    "            reservation_values={\"water\": 30, \"energy\": 10, \"land\": 20}\n",
    "        ),\n",
    "        policy=NegotiationPolicy(type=PolicyType.CONCEDER)\n",
    "    ),\n",
    "    Entity(\n",
    "        name=\"Industrial_Nation\",\n",
    "        utility_function=UtilityFunction(\n",
    "            weights={\"water\": 0.2, \"energy\": 0.6, \"land\": 0.2},\n",
    "            ideal_values={\"water\": 30, \"energy\": 70, \"land\": 40},\n",
    "            reservation_values={\"water\": 15, \"energy\": 35, \"land\": 20}\n",
    "        ),\n",
    "        policy=NegotiationPolicy(type=PolicyType.BOULWARE)\n",
    "    ),\n",
    "    Entity(\n",
    "        name=\"Tech_Nation\",\n",
    "        utility_function=UtilityFunction(\n",
    "            weights={\"water\": 0.2, \"energy\": 0.5, \"land\": 0.3},\n",
    "            ideal_values={\"water\": 25, \"energy\": 60, \"land\": 35},\n",
    "            reservation_values={\"water\": 10, \"energy\": 30, \"land\": 15}\n",
    "        ),\n",
    "        policy=NegotiationPolicy(type=PolicyType.ADAPTIVE)\n",
    "    )\n",
    "]\n",
    "\n",
    "# Run with simultaneous protocol (better for multi-party)\n",
    "multi_config = SimulationConfig(\n",
    "    entities=countries,\n",
    "    issues=resource_issues,\n",
    "    max_rounds=100,\n",
    "    protocol=\"simultaneous\"\n",
    ")\n",
    "\n",
    "multi_engine = NegotiationEngine(multi_config)\n",
    "multi_outcome = multi_engine.run()\n",
    "\n",
    "print(multi_outcome.summary())\n",
    "\n",
    "if multi_outcome.success:\n",
    "    print(\"\\nResource Allocation:\")\n",
    "    for resource, amount in multi_outcome.final_agreement.items():\n",
    "        print(f\"  {resource}: {amount:.1f}\")\n",
    "    \n",
    "    print(\"\\nCountry Satisfaction:\")\n",
    "    for country, utility in multi_outcome.final_utilities.items():\n",
    "        bar = \"█\" * int(utility * 10) + \"░\" * (10 - int(utility * 10))\n",
    "        print(f\"  {country:20} {bar} {utility*100:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Parameter Optimization Loop"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Start with suboptimal parameters\n",
    "buyer_opt = Entity(\n",
    "    name=\"Buyer\",\n",
    "    utility_function=UtilityFunction(\n",
    "        weights={\"price\": 0.7, \"quantity\": 0.3},\n",
    "        ideal_values={\"price\": 1000, \"quantity\": 1000},\n",
    "        reservation_values={\"price\": 4000, \"quantity\": 200}\n",
    "    ),\n",
    "    policy=NegotiationPolicy(\n",
    "        type=PolicyType.LINEAR_CONCESSION,\n",
    "        params=PolicyParameters(\n",
    "            accept_threshold=0.9,  # Too high!\n",
    "            concession_rate=0.02   # Too slow!\n",
    "        )\n",
    "    )\n",
    ")\n",
    "\n",
    "seller_opt = Entity(\n",
    "    name=\"Seller\",\n",
    "    utility_function=UtilityFunction(\n",
    "        weights={\"price\": 0.8, \"quantity\": 0.2},\n",
    "        ideal_values={\"price\": 5000, \"quantity\": 100},\n",
    "        reservation_values={\"price\": 2000, \"quantity\": 800}\n",
    "    ),\n",
    "    policy=NegotiationPolicy(\n",
    "        type=PolicyType.BOULWARE,\n",
    "        params=PolicyParameters(\n",
    "            accept_threshold=0.85,  # Also too high!\n",
    "            stubbornness=0.9       # Too stubborn!\n",
    "        )\n",
    "    )\n",
    ")\n",
    "\n",
    "opt_config = SimulationConfig(\n",
    "    entities=[buyer_opt, seller_opt],\n",
    "    issues=issues,\n",
    "    max_rounds=50\n",
    ")\n",
    "\n",
    "# Run optimization loop\n",
    "print(\"Optimization Progress:\")\n",
    "print(\"=\"*50)\n",
    "\n",
    "optimized_config, history = analyze_and_improve(opt_config, n_iterations=5)\n",
    "\n",
    "# Show improvement\n",
    "for i, outcome in enumerate(history):\n",
    "    status = \"✅ SUCCESS\" if outcome.success else \"❌ FAILED\"\n",
    "    avg_utility = np.mean(list(outcome.final_utilities.values()))\n",
    "    print(f\"Iteration {i+1}: {status} - Avg Utility: {avg_utility:.3f}\")\n",
    "\n",
    "print(\"\\nFinal Parameters:\")\n",
    "for entity in optimized_config.entities:\n",
    "    print(f\"\\n{entity.name}:\")\n",
    "    print(f\"  Accept Threshold: {entity.policy.params.accept_threshold:.2f}\")\n",
    "    print(f\"  Concession Rate: {entity.policy.params.concession_rate:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Sensitivity Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test sensitivity to acceptance threshold\n",
    "thresholds = np.linspace(0.5, 0.9, 9)\n",
    "threshold_results = []\n",
    "\n",
    "for threshold in thresholds:\n",
    "    # Set both entities to same threshold\n",
    "    buyer.policy.params.accept_threshold = threshold\n",
    "    seller.policy.params.accept_threshold = threshold\n",
    "    \n",
    "    config = SimulationConfig(\n",
    "        entities=[buyer, seller],\n",
    "        issues=issues,\n",
    "        max_rounds=50\n",
    "    )\n",
    "    \n",
    "    # Run small batch\n",
    "    runner = BatchNegotiationRunner(config)\n",
    "    results = runner.run_batch(10)\n",
    "    \n",
    "    success_rate = sum(1 for r in results if r.success) / len(results)\n",
    "    avg_rounds = np.mean([r.rounds_taken for r in results])\n",
    "    \n",
    "    threshold_results.append({\n",
    "        'threshold': threshold,\n",
    "        'success_rate': success_rate,\n",
    "        'avg_rounds': avg_rounds\n",
    "    })\n",
    "\n",
    "# Plot results\n",
    "df_sensitivity = pd.DataFrame(threshold_results)\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))\n",
    "\n",
    "ax1.plot(df_sensitivity['threshold'], df_sensitivity['success_rate'], 'o-')\n",
    "ax1.set_xlabel('Acceptance Threshold')\n",
    "ax1.set_ylabel('Success Rate')\n",
    "ax1.set_title('Success Rate vs Acceptance Threshold')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "ax2.plot(df_sensitivity['threshold'], df_sensitivity['avg_rounds'], 'o-')\n",
    "ax2.set_xlabel('Acceptance Threshold')\n",
    "ax2.set_ylabel('Average Rounds')\n",
    "ax2.set_title('Negotiation Length vs Acceptance Threshold')\n",
    "ax2.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nOptimal threshold appears to be around:\", \n",
    "      df_sensitivity.loc[df_sensitivity['success_rate'].idxmax(), 'threshold'])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Export Configuration for CLI"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save the configuration to YAML for use with CLI\n",
    "import yaml\n",
    "\n",
    "# Convert config to YAML format\n",
    "yaml_config = {\n",
    "    'entities': [\n",
    "        {\n",
    "            'name': buyer.name,\n",
    "            'type': buyer.type,\n",
    "            'utility': {\n",
    "                'weights': buyer.utility_function.weights,\n",
    "                'ideal_values': buyer.utility_function.ideal_values,\n",
    "                'reservation_values': buyer.utility_function.reservation_values\n",
    "            },\n",
    "            'policy': {\n",
    "                'type': buyer.policy.type.value,\n",
    "                'params': buyer.policy.params.dict()\n",
    "            }\n",
    "        },\n",
    "        {\n",
    "            'name': seller.name,\n",
    "            'type': seller.type,\n",
    "            'utility': {\n",
    "                'weights': seller.utility_function.weights,\n",
    "                'ideal_values': seller.utility_function.ideal_values,\n",
    "                'reservation_values': seller.utility_function.reservation_values\n",
    "            },\n",
    "            'policy': {\n",
    "                'type': seller.policy.type.value,\n",
    "                'params': seller.policy.params.dict()\n",
    "            }\n",
    "        }\n",
    "    ],\n",
    "    'issues': [\n",
    "        {\n",
    "            'name': issue.name,\n",
    "            'min_value': issue.min_value,\n",
    "            'max_value': issue.max_value,\n",
    "            'unit': issue.unit\n",
    "        }\n",
    "        for issue in issues\n",
    "    ],\n",
    "    'max_rounds': 50,\n",
    "    'protocol': 'alternating'\n",
    "}\n",
    "\n",
    "# Save to file\n",
    "with open('my_negotiation.yaml', 'w') as f:\n",
    "    yaml.dump(yaml_config, f, default_flow_style=False)\n",
    "\n",
    "print(\"Configuration saved to my_negotiation.yaml\")\n",
    "print(\"\\nYou can now run from command line:\")\n",
    "print(\"  python cli.py run my_negotiation.yaml --viz\")\n",
    "print(\"  python cli.py batch my_negotiation.yaml --n-runs 100\")\n",
    "print(\"  python cli.py analyze my_negotiation.yaml\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "This notebook demonstrated:\n",
    "1. Basic two-party negotiations\n",
    "2. AI advisory recommendations\n",
    "3. Batch analysis of different strategies\n",
    "4. Multi-party resource allocation\n",
    "5. Parameter optimization\n",
    "6. Sensitivity analysis\n",
    "7. Export for CLI usage\n",
    "\n",
    "Next steps:\n",
    "- Try different policy combinations\n",
    "- Experiment with more complex utility functions\n",
    "- Add coalition dynamics\n",
    "- Integrate with LLM for natural language strategies"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}